In [11]:
#import qiskit tools
import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, Aer, IBMQ
from qiskit.tools.visualization import circuit_drawer
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise

#import python stuff
import matplotlib.pyplot as plt
import numpy as np
import time

# Set backend device, choose to use a simulator
sim = Aer.get_backend('aer_simulator')

# functions..............................................................................................
def make_chsh_circuit(theta_vec):
    """Return a list of QuantumCircuits for use in a CHSH experiemnt
    (one circuit for each value of theta in theta_vec)
    (theta is the angle between the bases of Alice and Bob)
    
        Args:
            theta_vec (list): list of values of angles between the bases of Alice and Bob
        
        Returns:
            List[QuantumCircuit]: CHSH QuantumCircuits for each value of theta
    """
    chsh_circuits = []
    
    for theta in theta_vec:
        obs_vec = ['00', '01', '10', '11'] # observed_vector? ex: '01' means 1st measurement got 0, second got 1
        for el in obs_vec:
            qc = QuantumCircuit(2,2) # create a quantum circuit with two quantum register(qubits) and two classical register
            qc.h(0) # add a H gate on qubit 0
            qc.cx(0, 1) # Add a CX (CNOT) gate on control qubit 0 and target qubit 1
            qc.ry(theta, 0) # rotate around y-axis by theta
            for a in range(2): # what does this do? why do we need this part?
                if el[a] == '1':
                    qc.h(a) 
            qc.measure(range(2),range(2)) # measure q0 save to bit_0, then measure q1 save to bit_1
            chsh_circuits.append(qc)

    return chsh_circuits

def compute_chsh_witness(counts):
    """Computes expectation values for the CHSH inequality, for each
    angle (theta) between measurement axis.

        Args: counts (list[dict]): dict of counts for each experiment
              (4 per value of theta)

        Returns:
            Tuple(List, List): Tuple of lists with the two CHSH witnesses
    """
    # Order is ZZ,ZX,XZ,XX
    
    CHSH1 = []
    CHSH2 = []
    # Divide counts(list of dictionaries) in sets of 4
    for i in range(0, len(counts), 4):  
        theta_dict = counts[i:i + 4]
        zz = theta_dict[0] # alice uses z basis, bob uses z basis, which is the same as a and b measurement.
        zx = theta_dict[1] # a and b'
        xz = theta_dict[2]
        xx = theta_dict[3]
        print(zz)

        no_shots = sum(xx[y] for y in xx)

        chsh1 = 0
        chsh2 = 0

        # E(a,b)
        for element in zz: # zz is a dict
            parity = (-1)**(int(element[0])+int(element[1])) # plus for N_00, N_11. minus for N_01, N_10
            chsh1+= parity*zz[element]
            chsh2+= parity*zz[element]

        # E(a,b')
        for element in zx:
            parity = (-1)**(int(element[0])+int(element[1]))
            chsh1+= parity*zx[element]
            chsh2-= parity*zx[element]

        # E(a',b)
        for element in xz:
            parity = (-1)**(int(element[0])+int(element[1]))
            chsh1-= parity*xz[element]
            chsh2+= parity*xz[element]

        # E(a',b')
        for element in xx:
            parity = (-1)**(int(element[0])+int(element[1]))
            chsh1+= parity*xx[element]
            chsh2+= parity*xx[element]

        CHSH1.append(chsh1/no_shots)
        CHSH2.append(chsh2/no_shots)
    
    return CHSH1 #return the S of each circuit
# functions...........................................................................................

#split 0 to 2pi into 15 angles. use these angles to build the chsh circuit
number_of_thetas = 15
theta_vec = np.linspace(0,2*np.pi,number_of_thetas)
my_chsh_circuits = make_chsh_circuit(theta_vec)

# test and see the circuit: the circuits create a bell pair, and measure them in different basis. 
# Bob(q1) in +,-,0,1 state, Alice(q0) in theta respected to Bob
# my_chsh_circuits[5].draw()

# Execute and get counts
num_of_shots = 10000
result_ideal = sim.run(my_chsh_circuits, shots=num_of_shots).result()
counts_ideal = result_ideal.get_counts()
CHSH1_ideal = compute_chsh_witness(counts_ideal) # this is the S value

print(counts_ideal)  


[{'00': 4854, '11': 5146}, {'01': 2497, '00': 2439, '10': 2520, '11': 2544}, {'01': 2493, '11': 2485, '00': 2526, '10': 2496}, {'00': 5057, '11': 4943}, {'10': 237, '00': 4752, '01': 250, '11': 4761}, {'01': 3570, '11': 1441, '00': 1400, '10': 3589}, {'11': 3617, '01': 1360, '10': 1417, '00': 3606}, {'01': 242, '11': 4769, '10': 252, '00': 4737}, {'01': 925, '00': 4080, '10': 964, '11': 4031}, {'01': 4433, '11': 544, '00': 523, '10': 4500}, {'01': 534, '00': 4464, '10': 562, '11': 4440}, {'01': 955, '11': 4073, '00': 3968, '10': 1004}, {'01': 1996, '11': 3099, '00': 3042, '10': 1863}, {'11': 76, '01': 5052, '00': 64, '10': 4808}, {'01': 74, '11': 4977, '10': 65, '00': 4884}, {'00': 3003, '10': 1932, '11': 3093, '01': 1972}, {'01': 3002, '00': 1933, '10': 3055, '11': 2010}, {'11': 59, '00': 71, '10': 4848, '01': 5022}, {'01': 62, '10': 58, '00': 4932, '11': 4948}, {'00': 1909, '10': 3054, '11': 1948, '01': 3089}, {'11': 900, '00': 940, '10': 4080, '01': 4080}, {'11': 569, '01': 4457, '0

In [12]:
alice = []
alice_ratio = []

for i in range(0,len(counts_ideal)):
    if ('00' in counts_ideal[i]) == False:
        counts_ideal[i]['00'] = 0
    if ('01' in counts_ideal[i]) == False:
        counts_ideal[i]['01'] = 0
    if ('10' in counts_ideal[i]) == False:
        counts_ideal[i]['10'] = 0
    if ('11' in counts_ideal[i]) == False:
        counts_ideal[i]['11'] = 0
   
    a_temp = {} # {} to declare a dict
    a0 = counts_ideal[i]['00'] + counts_ideal[i]['01']
    a1 = counts_ideal[i]['10'] + counts_ideal[i]['11']
    a_temp['0'] = a0
    a_temp['1'] = a1
    alice.append(a_temp)
    
    r = a0/a1
    alice_ratio.append(r)
        
print(alice_ratio)
    

[0.9432568985619899, 0.9747235387045814, 1.0076289901626179, 1.0230629172567267, 1.0008003201280513, 0.9880715705765407, 0.9864918553833929, 0.9916351324437364, 1.002002002002002, 0.9825535289452815, 0.9992003198720512, 0.9696671262556628, 1.015316404675534, 1.0475020475020476, 0.9833399444664815, 0.9900497512437811, 0.9743336623889437, 1.037905033625433, 0.9976028765481423, 0.9992003198720512, 1.0080321285140563, 1.0157226365652086, 1.0100502512562815, 1.014098690835851, 0.9821605550049554, 0.9940179461615155, 0.9755037534571316, 0.998001998001998, 0.9896538002387585, 0.9860973187686196, 1.0036064916850331, 0.9813750743015652, 0.9798059790140566, 0.9821605550049554, 0.994415636218588, 0.9860973187686196, 0.9813750743015652, 0.9984012789768185, 1.0132876988121602, 1.008838891120932, 1.0145044319097503, 0.9940179461615155, 1.0004000800160031, 0.9984012789768185, 0.994415636218588, 0.9708316909735909, 1.0036064916850331, 1.0366598778004072, 1.0064205457463884, 1.0292207792207793, 1.02593

In [13]:
bob = []
bob_ratio = []

for i in range(0,len(counts_ideal)):
    if ('00' in counts_ideal[i]) == False:
        counts_ideal[i]['00'] = 0
    if ('01' in counts_ideal[i]) == False:
        counts_ideal[i]['01'] = 0
    if ('10' in counts_ideal[i]) == False:
        counts_ideal[i]['10'] = 0
    if ('11' in counts_ideal[i]) == False:
        counts_ideal[i]['11'] = 0
   
    b_temp = {} # {} to declare a dict
    b0 = counts_ideal[i]['00'] + counts_ideal[i]['10']
    b1 = counts_ideal[i]['01'] + counts_ideal[i]['11']
    b_temp['0'] = b0
    b_temp['1'] = b1
    bob.append(a_temp)
    
    r = b0/b1
    bob_ratio.append(r)
        
print(bob_ratio)

[0.9432568985619899, 0.9837333862328903, 1.008838891120932, 1.0230629172567267, 0.9956096587507484, 0.9956096587507484, 1.0092425155716296, 0.9956096587507484, 1.0177562550443906, 1.0092425155716296, 1.010454362685967, 0.9888623707239459, 0.9627085377821394, 0.9500780031201248, 0.9798059790140566, 0.9743336623889437, 0.9952114924181963, 0.9681165124975398, 0.9960079840319361, 0.9853087155052611, 1.0080321285140563, 0.9896538002387585, 1.0132876988121602, 1.0004000800160031, 1.0128824476650564, 1.0234722784297856, 0.9996000799840032, 1.011667672500503, 1.010454362685967, 0.9817677368212445, 0.9952114924181963, 1.0189783969311528, 1.02757502027575, 0.9786307874950534, 0.9747235387045814, 1.0181634712411705, 1.0157226365652086, 0.9821605550049554, 1.012477359629704, 0.9758940920766647, 0.9747235387045814, 0.9924287706714485, 0.9928258270227183, 1.0292207792207793, 0.9857029388403494, 0.9723865877712031, 0.9976028765481423, 1.0420665713702266, 0.9936204146730463, 0.9936204146730463, 0.9673